# 需求描述
1. 国家是否扶持
    * 找industry (boolean)
    * 更改后续权重
3. 市净率
    * 小于20%
4. 是否盈利
    * 是
5. 找市值
    * 超过100亿
    * 不看超过2500

In [2]:
# Load dependencies
import yfinance as yf
import math
# I HATE PYTHON IMPORT SO MUCH. DUPLICATING helpers.py here so I don't have to deal with it.
from helpers import load_txt

In [3]:
# 代码、名称、细分产业、所属行业、市净率、每股盈利、总市值、净利润?
stocks = load_txt('../experiment_3/tables2')
stocks.head()

,代码,名称,总市值,流通市值,净利润?,市净率,每股盈利,细分行业,所属行业
0,600000.SS,浦发银行,253896230000,253896230000,41536000000,0.459,1.34,股份制银行,银行
1,600004.SS,白云机场,28329618000,24769767000,-490145410,1.550,-0.210,机场,机场航运
2,600006.SS,东风汽车,13940000000,13940000000,513990960,1.710,0.257,商用载货车,汽车整车
3,600007.SS,中国国贸,14484723000,14484723000,775740970,1.730,0.770,商业地产,房地产开发
4,600008.SS,首创环保,23636702000,23636702000,1914004000,1.360,0.261,综合环境治理,环保


## 国家是否扶持

In [5]:
def industry_check_yh(stock, dict, ind=['Electronic Components', 'Metal Fabrication']):
    dict['industry'] = stock.info['industry']
    return stock.info['industry'] in ind

def industry_check_ths(stock, ind=[]):
    return stock['细分行业'] in ind

# industry_check_ths(stocks[stocks['代码'] == '600435.SS'])

False

## 市净率 (P/B Ratio)

In [13]:
def pb_check_yh(stock, dict, cutoff=20):
    if 'priceToBook' in stock.info:
        dict['priceToBook'] = stock.info['priceToBook']
        return stock.info['priceToBook'] < cutoff
    else:
        print('错误：数据源未提供市净率')
        return False

def pb_check_ths(stock, cutoff=20):
    return stock['市净率'] < cutoff

# pb_check_ths(stocks[stocks['代码'] == '600435.SS'])

## 是否盈利

In [14]:
def profitable_check_yh(stock, dict, key='profitMargins'):
    if key in stock.info:
        dict[key] = stock.info[key]
        return stock.info['priceToBook']
    else:
        print('错误：数据源未提供盈利数据')
        return False

def profitable_check_ths(stock, key='净利润?'):
    return stock[key] != '--'

# profitable_check_ths(stocks[stocks['代码'] == '600435.SS'])

## 找市值

In [15]:
def  cap_check_yh(stock, dict, min=100, max=2500):
    yi = math.pow(10, 8) # 一亿
    if 'marketCap' in stock.info:
        dict['marketCap'] = stock.info['marketCap'] / yi
        return (stock.info['marketCap'] >= min * yi) & (stock.info['marketCap'] <= max * yi)
    else:
        print('错误：数据源未提供市值')
        return False

def cap_check_ths(stock, min=100, max=2500):
    yi = math.pow(10, 8) # 一亿
    if stock['总市值'] != '--':
        return (int(stock['总市值']) >= min * yi) & (int(stock['总市值']) <= max * yi)
    else:
        name = stock['    名称']
        print(f'数据源未提供{name}市值')
        return False

# cap_check_ths(stocks[stocks['代码'] == '600435.SS'])

In [39]:
# 基础分析
def fundamentals(industries=None, tickers=None, debug=False):
    # 代码、名称、细分行业、所属行业、市净率、每股盈利、总市值、净利润?
    stocks = load_txt('../experiment_3/tables2')
    qualified = []

    if industries == None:
        industries = stocks['细分行业'].unique()

    if tickers != None:
        stocks = stocks.loc[stocks['代码'].isin(tickers)]

    for index, row in stocks.iterrows():
        if check_stock_ths(row, industries, debug == True or tickers != None):
            qualified.append(row['代码'])
            name = row['    名称']
            ticker = row['代码']
            print(f'{name}({ticker})满足长线条件...')

    return qualified

def check_stock_yh(stock, debug=False):
    out = {}
    if industry_check_yh(stock, out) == False:
        if debug: print('%s的行业不符合条件'%stock.ticker)
        return False
    elif pb_check_yh(stock, out) == False:
        if debug: print('%s的市净率不符合条件'%stock.ticker)
        return False
    elif profitable_check_yh(stock, out) == False:
        if debug: print('%s的当前未盈利'%stock.ticker)
        return False
    elif cap_check_yh(stock, out) == False:
        if debug: print('%s的市值不符合条件'%stock.ticker)
        return False

    if debug: print('%满足长线条件...'%stock.info['shortName'])
    return out

def check_stock_ths(stock, industries, output=False):
    if industry_check_ths(stock, industries) == False:
        if output: print('%s的产业不符合条件'%stock['    名称'])
        return False
    elif pb_check_ths(stock) == False:
        if output: print('%s的市净率不符合条件'%stock['    名称'])
        return False
    elif profitable_check_ths(stock) == False:
        if output: print('%s的当前未盈利'%stock['    名称'])
        return False
    elif cap_check_ths(stock) == False:
        if output: print('%s的市值不符合条件'%stock['    名称'])
        return False
    return True

def show_industries():
    stocks = load_txt('../experiment_3/tables2')
    print(stocks['细分行业'].unique())

In [40]:
# show_industries()
fundamentals(['光伏设备'])

航天机电(600151.SS)满足长线条件...
通威股份(600438.SS)满足长线条件...
爱旭股份(600732.SS)满足长线条件...
洛阳玻璃(600876.SS)满足长线条件...
福莱特(601865.SS)满足长线条件...
上机数控(603185.SS)满足长线条件...
赛伍技术(603212.SS)满足长线条件...
金辰股份(603396.SS)满足长线条件...
福斯特(603806.SS)满足长线条件...
大全能源(688303.SS)满足长线条件...
中信博(688408.SS)满足长线条件...
奥特维(688516.SS)满足长线条件...
高测股份(688556.SS)满足长线条件...
金博股份(688598.SS)满足长线条件...
天合光能(688599.SS)满足长线条件...
海优新材(688680.SS)满足长线条件...
中环股份(002129.SZ)满足长线条件...
拓日新能(002218.SZ)满足长线条件...
晶澳科技(002459.SZ)满足长线条件...
协鑫集成(002506.SZ)满足长线条件...
爱康科技(002610.SZ)满足长线条件...
联泓新科(003022.SZ)满足长线条件...
易成新能(300080.SZ)满足长线条件...
东方日升(300118.SZ)满足长线条件...
阳光电源(300274.SZ)满足长线条件...
晶盛机电(300316.SZ)满足长线条件...
中来股份(300393.SZ)满足长线条件...
捷佳伟创(300724.SZ)满足长线条件...
帝尔激光(300776.SZ)满足长线条件...
上能电气(300827.SZ)满足长线条件...


['600151.SS',
 '600438.SS',
 '600732.SS',
 '600876.SS',
 '601865.SS',
 '603185.SS',
 '603212.SS',
 '603396.SS',
 '603806.SS',
 '688303.SS',
 '688408.SS',
 '688516.SS',
 '688556.SS',
 '688598.SS',
 '688599.SS',
 '688680.SS',
 '002129.SZ',
 '002218.SZ',
 '002459.SZ',
 '002506.SZ',
 '002610.SZ',
 '003022.SZ',
 '300080.SZ',
 '300118.SZ',
 '300274.SZ',
 '300316.SZ',
 '300393.SZ',
 '300724.SZ',
 '300776.SZ',
 '300827.SZ']